In [4]:
import torch
from torch import nn
import numpy as np

# Creating a line model

In [6]:
class LineNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(1, 1))

    def forward(self, x: list) -> nn.Sequential:
        return self.layers(x)

# Train our model

## Preparing our data set to train line model

In [32]:
from torch.utils.data import Dataset, DataLoader
import torch.distributions.uniform as urand

In [33]:
class AlgebricDataSet(Dataset):
    def __init__(self, f, interval: tuple, nsamples: int) -> None:
        X = urand.Uniform(interval[0], interval[1]).sample([nsamples])
        self.data = [(x, f(x)) for x in X]

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> tuple:
        return self.data[idx]

In [34]:
line = lambda x: 2 * x + 3
interval = (-10, 10)
train_nsamples = 1000
test_nsamples = 1000

In [35]:
train_dataset = AlgebricDataSet(line, interval, train_nsamples)
test_dataset = AlgebricDataSet(line, interval, test_nsamples)

In [36]:
train_dataloader = DataLoader(train_dataset, train_nsamples, shuffle=True)
test_dataloader = DataLoader(test_dataset, test_nsamples, shuffle=True)

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [39]:
model = LineNetwork().to(device)

In [41]:
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

ValueError: optimizer got an empty parameter list